[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/How-to-Learn-to-Code/python-class/blob/master/Lesson_5_Pandas_DataFrame/Lesson5_pandas_DataFrame_Teacher.ipynb)

# Lesson 5: Pandas as a dataframe API 

## Learning objectives

Students will gain a better understanding of how to use pandas for manipulating data frames

Specific coding skills learned:
- Subsetting data frames
- Joining data frames together 
- Other useful pandas functions (incuding groupby, statistics, and conditional subsetting)

In [ ]:
%pip install pandas 
%pip install matplotlib 
%pip install numpy 
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

### Introduction

In our last class, we discussed the basics of how to use pandas to read in our data. When we load in data from a CSV file with Pandas using a function such as reas_csv, we get a data structure known as a data frame. Basically, this is a two-dimensional table of rows and columns. 

This is useful in terms of allowing us to visualize our data, but most users will want to utilize the data for their own research purposes. Therefore, users will find it helpful to use pandas as an application programming interface (or API), which is basically a set of functions that allow users access to the features of the data. 

For this class, we are going to be using daily weather history data from multiple U.S. cities from July 2014 - June 2015 (downloaded from FiveThirtyEight's public repository of data found here: https://github.com/fivethirtyeight/data/tree/master/us-weather-history). 

The first set of data we are going to read in is from Philadelphia.

In [ ]:
!wget https://raw.githubusercontent.com/fivethirtyeight/data/master/us-weather-history/KPHL.csv
philly_weather = pd.read_csv('KPHL.csv',index_col='date')
philly_weather.head()

### Subsetting Data Frames

The rows of this data frame represent each day during this period, and the columns represent the weather data that has been collected.

As you can see, there are numerous types of data that has been collected, including the temperatures that occurred that day, what the average temperatures have been for that day over the years, the records for that day, and the amount of precipitation that occurred both on that day and on average for that day. 

##### Question 1: Let's say that you do not care about the temperature data, so most of these columns are not neccessary. How can you get the data you want to have? 

##### Answer: You can subset the data frame to only contain the columns you need

In pandas, one of the ways you can select certain subsets of the data is to specify them by the row and/or column names.  

To select a particular column (for example, the actual preciptation for each day) , you use square brackets [] along with the column name of the column of interest in quotation marks. Alternatively, this is known as "slicing", since you are taking a slice of the original data frame.

In [ ]:
philly_weather["actual_precipitation"]

In order to get both columns of precipitation data, we need to use double brackets, then within the brackets we list the columns of interest with a comma between them, as shown below. 

In [ ]:
philly_weather[["actual_precipitation","average_precipitation","record_precipitation"]]

Another way of selecting these columns is to add to the end of our data frame object ".loc" (which looks for the names of the columns by labels). Since our matrix is rows by columns, the ":" symbol tells the computer that we want all of the rows, but only the columns with specific labels. 

In [ ]:
philly_weather.loc[:,["actual_precipitation","average_precipitation","record_precipitation"]]

Likewise, we can do the same for the rows, only placing the ":" after the comma (in the columns position), and selecting the rows by label before the comma, as shown below. 

In [ ]:
philly_weather.loc[["2014-7-1"],:]

In [ ]:
philly_weather.loc[["2014-7-1","2014-7-2"],:]

While this is not too difficult if we only need a handful of rows and/or columns, this could become time-consuming if we want to get a series of rows and/or columns.

##### Question 2: Instead of using the labels of the rows/columns, what is another attribute you can use to subset the data? 

##### Answer: You can use their numerical location (ex: the 1st row or 2nd column) 

A little earlier in the lesson, we used ".loc" to select rows and columns by their label. To do this by their index, we can instead use ".iloc" to select the row and/or column index value(s). 

Given that the "actual_precipitation" and "average_precipitation" columns represent the 9th through 11th rows respectively, this is how we subset the data to obtain the values in these columns .

(note: the first piece of code means "select the data from the 9th column up to, but not including, the 12th column").

In [ ]:
philly_weather.iloc[:,9:12]

In [ ]:
philly_weather.iloc[0:2,:]

If the numbers are not in a sequence (ex: columns 0, 5, and 7), you can get the subset of the data like this

In [ ]:
philly_weather.iloc[:,[0,5,7]]

### Joining Data Frames 

Of course, we might not only be interested in the data from one city. Imagine that we want to do side-by-side comparisons of the average precipitation in Seattle vs. Philadelphia. 

Let's first read in the weather pattern data for Seattle as well.

In [ ]:
!wget https://raw.githubusercontent.com/fivethirtyeight/data/master/us-weather-history/KSEA.csv
seattle_weather = pd.read_csv('KSEA.csv',index_col='date')
seattle_weather.head()

Using what we learned earlier, we can subset our data to only look at the columns dealing with precipitation:

In [ ]:
philly_weather_precip = philly_weather.iloc[:,9:12]
seattle_weather_precip = seattle_weather.iloc[:,9:12]

Rather than having to flip between both data frames, we can instead combine them into one separate data frame using the function "join". 

There are four different ways that we can join our data, which we will go through below: "left", "right", "inner", and "outer" 

For the sake of an example, let us sort our data such that Philadelphia's precipitation data is sorted by the lowest to highest record precipitation, while Seattle's precipitation data is sorted from the highest to lowest record precipitation.

In [ ]:
philly_weather_precip_reordered = philly_weather_precip.sort_values(by=["record_precipitation"], axis=0, ascending=True)
philly_weather_precip_reordered

In [ ]:
seattle_weather_precip_reordered = seattle_weather_precip.sort_values(by=["record_precipitation"], axis=0, ascending=False)
seattle_weather_precip_reordered

A left join indicates that we preserve the order of the data frame is getting another data frame joined to it (in this case, the Philadelphia precipitation data). Since the suffix names are the same for both dataframes, we use lsuffix and rsuffix to specify which columns came from which dataframe. 

(Note: 'left' is the default setting)

In [ ]:
philly_weather_precip_reordered.join(seattle_weather_precip_reordered, how ='left', lsuffix = '_philly', rsuffix = '_seattle')

Likewise, a right join indicates that we preserve the order of the data frame that is being joined to the first data frame (in this case, the Seattle precipitation data).

In [ ]:
philly_weather_precip_reordered.join(seattle_weather_precip_reordered, how ='right', lsuffix = '_philly', rsuffix = '_seattle')

If we want to order the dates lexographically, we can change the how parameter to 'outer' (or the union of the datasets). 

In [ ]:
philly_weather_precip_reordered.join(seattle_weather_precip_reordered, how ='outer', lsuffix = '_philly', rsuffix = '_seattle')

When we combine these datasets together, we can easily compare data from both data frames, for example plotting the record precipitation in Seattle vs. Philadelphia

In [ ]:
all_precipitation = philly_weather_precip.join(seattle_weather_precip,lsuffix = '_philly', rsuffix = '_seattle')
ax = plt.gca()
all_precipitation.plot(kind='line',y='record_precipitation_philly',ax=ax, color = 'purple',figsize=(12,5))
all_precipitation.plot(kind='line',y='record_precipitation_seattle',ax=ax, color = 'green',figsize=(12,5))
plt.show()

Let us pretend that we want to add information about each month (stored in the months object)

In [ ]:
months = pd.Series(['July', 'August','September','October','November','December','January','February','March','April','May','June'])
months = months.repeat([31,31,30,31,30,31,31,28,31,30,31,30])
months = pd.DataFrame(months, columns = ['months'])
months.index = all_precipitation.index
months.index.name = 'date'
months

Now, let us say that in addition to the rainfall data, we also started to collect information about the average dewpoint on each day for the first three months in Philadelphia. However, due to statewide budget cuts scientists could not afford to gather data for the rest of the year.

In [ ]:
dewpoint = np.random.randint(40,70,size=92)
dewpoint = pd.DataFrame(dewpoint, columns=['dewpoint'])
dewpoint.index = all_precipitation.index[0:92]
dewpoint.index.name = 'date'
dewpoint

As done previously, we can join the datasets together, but you will notice below that all of the dates where there was no dewpoint recorded are labeled 'NaN' ("Not a number", or undefined, data points).

In [ ]:
philly_weather_precip.join(dewpoint)

What if you only want to keep the values where dewpoints are recorded? To do this, you can use join with the how parameter set to 'inner' (or intersection of the data), as shown below. 

In [ ]:
philly_weather_precip.join(dewpoint, how='inner')

### Other useful pandas functions: conditional subsetting 

There are other ways to subset datasets by their index or their rownames/colnames. 

Let us say that you are only looking for days where the max temperature was greater than 90. How would we be able to get only those rows?

To do this, we can do something called conditional subsetting.

Remember in lesson 1 when we learned about comparisons, such as "greater than" ( > ), "lesser than" ( < ),  "equals" (==)? We can subset our data this way as well. 

To pick the rows where the max temperature was greater than 90, we can subset the data as follows

In [ ]:
philly_weather[philly_weather["actual_max_temp"] > 90]

As another example, what if we wanted to get all the days where it did not rain?

In [ ]:
philly_weather[philly_weather["actual_precipitation"] == 0.00]

What if we want to omit years where the high occurred that year? (Hint: remember the boolean operators AND (&) and OR (|)

Answer:

In [ ]:
philly_weather[(philly_weather["record_max_temp_year"] != 2014) & (philly_weather["record_max_temp_year"] != 2015)]

### Other useful pandas functions: groupby

Let us go back to to the all_precip_wmonths object, when we added information about which month each day was in for the combined Philly and Seattle precipitation data. One neat thing we can do is to group data based on certain column values (such as months), as shown below. 

In [ ]:
all_precip_wmonths.groupby(["months"])

We can do functions such as mean, min, and max for each month, rather than every day. 

In [ ]:
all_precip_wmonths.groupby(["months"]).mean()

In [ ]:
all_precip_wmonths.groupby(["months"]).min()

In [ ]:
all_precip_wmonths.groupby(['months']).max()

Remember when we plotted the record precipitation in Philadelphia vs. Seattle? What if we instead plotted the mean month-to-month instead of daily?

In [ ]:
mean_months_precip = all_precip_wmonths.groupby(["months"]).mean()
ax2 = plt.gca()
mean_months_precip.plot(kind='line',y='record_precipitation_philly',ax=ax2, color = 'purple',figsize=(12,5))
mean_months_precip.plot(kind='line',y='record_precipitation_seattle',ax=ax2, color = 'green',figsize=(12,5))
plt.show()

From this graph, we can see that months where the record precipitation was on average the highest in Philadelphia (such as August and July) are the lowest precipitation times in Seattle!

### Other useful pandas functions: statistics

Lastly, we can perform statistics on our data as a whole. We have already gone through how certain mathematical functions such as min, max, or mean can be calculated on different groups, but this can also be done to columns of the data frame as well. 

For example, we can get the mean value of the actual precipitation in Seattle and Philadelphia, as shown below:

In [ ]:
all_precip_wmonths["actual_precipitation_seattle"].mean()

In [ ]:
all_precip_wmonths["actual_precipitation_philly"].mean()

As you can see, on average the daily precipitation in Philadelphia is only 0.02 inches greater than Seattle. 

If you wanted to look at both at the same time, we follow the same rules that we did in label-based subsetting:

In [ ]:
all_precip_wmonths[["actual_precipitation_seattle","actual_precipitation_philly"]].mean()

In general, if you wanted to look at multiple statistics at once, you can call the function 'describe', as shown below:

In [ ]:
all_precip_wmonths[["actual_precipitation_seattle","actual_precipitation_philly"]].describe()

## In-class exercises

##### In-class exercise 1: How would we select the precipitation data from July 1st and July 2nd, 2014, in Philadelphia, using label-based subsetting?

##### Answer:

In [ ]:
philly_weather.loc[["2014-7-1","2014-7-2"],["actual_precipitation","average_precipitation","record_precipitation"]]

##### In-class exercise 2: What would we do to only select the precipitation data from July 1st and July 2nd, 2014 using index-based subsetting?

##### Answer:

In [ ]:
philly_weather.iloc[0:2,9:12]

##### In-class exercise 3: Using the 'all_precipitation' and 'months' objects called earlier, how can we join these dataframes together and save it as a new object titled 'all_precip_wmonths'?

##### Answer:

In [ ]:
all_precip_wmonths = all_precipitation.join(months)
all_precip_wmonths

##### In-class exercise 4: Go back and use the philly_weather and seattle_weather objects to figure out which days had a record minimum tempeature under 10 in either dataset

##### Answer: 

In [ ]:
all_weather = philly_weather.join(seattle_weather,lsuffix = '_philly', rsuffix = '_seattle')
all_weather[(all_weather["record_min_temp_philly"] < 10) | (all_weather["record_min_temp_seattle"] < 10)]

##### In-class exercise 5: Using the all_precip_wmonths object, how would you find the differences in standard deviation in average precipitation between Philadelphia and Seattle for each month? (Hint: you may want to use the pandas function "values" at  some point in this answer)

##### Answer:

In [ ]:
standard_dev_both = all_precip_wmonths[["actual_precipitation_philly","actual_precipitation_seattle","months"]].groupby(["months"]).std()
standard_dev_philly = all_precip_wmonths[["actual_precipitation_philly","months"]].groupby(["months"]).std()
standard_dev_seattle = all_precip_wmonths[["actual_precipitation_seattle","months"]].groupby(["months"]).std()
standard_dev_philly.values - standard_dev_seattle.values

## Homework 

There are 81 different exercises pertaining to manipulating Pandas data frames found here: https://www.w3resource.com/python-exercises/pandas/index-dataframe.php (with the predicted output as well as possible solutions to the problems). 

I would suggest doing problems 3, 4, 5, 7, 9, 10, 12, 13, 14, 15, 24, and 31 (and if you have time/interest, problems 33, 44, 45, 49, and 52 are also good choices)